<a href="https://colab.research.google.com/github/rrl7012005/Tensor-Flow-Computer-Vision-Course/blob/main/Human_Emotions_Detection_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  import opendatasets as od
except:
  !pip install opendatasets
  import opendatasets as od


import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import time
import random
from google.colab import files
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import matplotlib.cm as cm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers  import L2, L1
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature
from google.colab import drive

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
od.download("https://www.kaggle.com/datasets/muhammadhananasghar/human-emotions-datasethes")

Dataset URL: https://www.kaggle.com/datasets/muhammadhananasghar/human-emotions-datasethes


100%|██████████| 309M/309M [00:19<00:00, 17.0MB/s]


In [ ]:
CONFIGURATION = {
    "BATCH_SIZE": 32,
    "IM_SIZE": 256,
    "LEARNING_RATE": 1e-3,
    "N_EPOCHS": 3,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASSES": 3,
    "PATCH_SIZE": 16,
    "PROJ_DIM": 768,
    "CLASS_NAMES": ["angry", "happy", "sad"],
}

train_directory = "/content/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/train"
val_directory = "/content/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/test"

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
)


val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=1,#CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
)

training_dataset = (
    train_dataset
    .prefetch(tf.data.AUTOTUNE)
)

val_dataset = (
    val_dataset
    .prefetch(tf.data.AUTOTUNE)
)

Found 6799 files belonging to 3 classes.
Found 2278 files belonging to 3 classes.


In [ ]:
from transformers import TFViTModel

class ViTLayer(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.vit = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

    def call(self, inputs):
        return self.vit(inputs, training=False).last_hidden_state[:, 0, :]

resize_rescale_hf = tf.keras.Sequential([
    Resizing(224, 224),
    Rescaling(1./255),
    Permute((3, 1, 2))
])

inputs = Input(shape=(256, 256, 3))
x = resize_rescale_hf(inputs)
x = ViTLayer()(x)
output = Dense(CONFIGURATION["NUM_CLASSES"], activation='softmax')(x)

hf_model = tf.keras.Model(inputs=inputs, outputs=output)
hf_model.summary()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 3, 224, 224)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vi_t_layer (ViTLayer)                │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 3)                   │           2,307 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,307 (9.01 KB)

 Trainable params: 2,307 (9.01 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
loss_function = CategoricalCrossentropy()

metrics = [CategoricalAccuracy(name = "accuracy"), TopKCategoricalAccuracy(k=2, name = "top_k_accuracy")]

optimizer = Adam(learning_rate=CONFIGURATION["LEARNING_RATE"])

hf_model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

history = hf_model.fit(training_dataset, validation_data=val_dataset, epochs=CONFIGURATION["N_EPOCHS"], verbose=1)

Epoch 1/3
213/213 ━━━━━━━━━━━━━━━━━━━━ 194s 823ms/step - accuracy: 0.5880 - loss: 0.8674 - top_k_accuracy: 0.8421 - val_accuracy: 0.7511 - val_loss: 0.6086 - val_top_k_accuracy: 0.9263
Epoch 2/3
213/213 ━━━━━━━━━━━━━━━━━━━━ 117s 552ms/step - accuracy: 0.7461 - loss: 0.6027 - top_k_accuracy: 0.9285 - val_accuracy: 0.7726 - val_loss: 0.5566 - val_top_k_accuracy: 0.9328
Epoch 3/3
213/213 ━━━━━━━━━━━━━━━━━━━━ 118s 556ms/step - accuracy: 0.7635 - loss: 0.5641 - top_k_accuracy: 0.9358 - val_accuracy: 0.7818 - val_loss: 0.5319 - val_top_k_accuracy: 0.9372


In [ ]:
hf_model.evaluate(val_dataset)

2278/2278 ━━━━━━━━━━━━━━━━━━━━ 39s 17ms/step - accuracy: 0.7917 - loss: 0.5070 - top_k_accuracy: 0.9386


[0.5062639117240906, 0.7906057834625244, 0.9420544505119324]

#Model Optimization

##ONNX

We can convert our trained tensorflow model to an onnx format and do inference on that format. The ONNX format can allow us to convert models built in tensorflow to a model usable in PyTorch and vice versa. This is useful if the implementation of a certain network is more efficient in one framework compared to another.

To convert to onnx format lets first save the model then install tf2onnx and onnxruntime tools. The write a command to conver the saved model into an onnx file

In [ ]:
hf_model.save('vit_finetuned.keras')

In [ ]:
!pip install -U tf2onnx
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.5 MB/s eta 0:00:00


Note the onnx format has a smaller size than just saving the model.

We can also convert keras files .h5 to onnx format

In [ ]:
hf_model.save('vit_finetuned.h5')

In [ ]:
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec(
    (None, CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3),
    tf.float32,
    name="input"),)

output_path = "vit_keras.onnx"

model_proto, _ = tf2onnx.convert.from_keras(hf_model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

In [ ]:
print(output_names)

['dense']


Now do inference with this onnx format. The provider allows this to be run on the CPU. Now you dont even need tensorflow anymore just onnx runtime

In [ ]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)

test_image = cv2.imread("/content/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/train/happy/387249.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))

im = test_image.astype(np.float32)
im = np.expand_dims(im, axis = 0)

Compare to our model, also lets check which method is faster. Recall tensorflow is on the gpu and onnx is on the cpu so there will be a difference.

In [ ]:
%%time
onnx_pred = m.run(output_names, {"input": im}) #im is our input image

CPU times: user 390 ms, sys: 1.99 ms, total: 392 ms
Wall time: 392 ms


In [ ]:
%%time
print(hf_model(im))

tf.Tensor([[0.13168773 0.8274541  0.04085813]], shape=(1, 3), dtype=float32)
CPU times: user 776 ms, sys: 14 ms, total: 790 ms
Wall time: 1.3 s


Lets install the gpu version of onnx

In [ ]:
!pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 6.1 MB/s eta 0:00:00


Now re run the above. Onnx GPU is much faster. We can put the cuda execution provider before the cpu execution provider so we only use the cpu execution provider if the cuda version is not available.

To get a better estimate, run the inference on the same input 10 times and take the average.

In [ ]:
providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)

test_image = cv2.imread("/content/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/train/happy/387249.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))

im = test_image.astype(np.float32)
im = np.expand_dims(im, axis = 0)

##Model Quantization

This is a technique for performing computations and storing tensors at lower bit widths. The network parameters and activations are cconverted from floating point representation to lower precision representation of 8 bit integers typically.

Essentially can run models by using less memory space.

To go from float 32 into 8 bit integers we just do a simple linear mapping from typically -1 to 1 (as most weights are in that range) to signed integers in the range -128 to 128 or unsigned integers in the range 0 to 255. So we lose some precision and also some accuracy but probably not by much, in return the memory used reduces by a lot, and thus the speed of inference.

There are several types of quantization.

In dynamic quantization,the quantization parameters are computed dynamically, on the fly, so the cost of inference is higher but we achieve higher accuracy. Here the weights are quantized before hand but the activations (output of layers) are not quantized.

In static quantization, the model is run on a set of small inputs called calibration data. During these runs the most appropriate quantization parameters are chosen and then used for all future inputs. Here the weights and the output of layers are quantized.

These 2 methods are post quantization methods, the model is trained on float 32 then converted to int8. Now post training quantization (PTQ) may not achieve acceptable accuracy so then QAT may be used, i.e we include the quantization error in the training phase, so the network can adapt to the quantized weights. Fake quantization operations are inserted into the training graph to simulate quantization of parameters and data, theyre called fake because they quantize the data and then immediatelt dequantize so the operations remains in float32. This adds quantization noise without changing much in the deep learning framework.

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

#Model and quantized file names
model_fp32 = '/content/vit_keras.onnx'
model_quant = '/content/vit_quantized.onnx'

quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type = QuantType.QUInt8)


The model is now 83MB

Now retime the predictions

In [ ]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(model_quant, providers=providers)

test_image = cv2.imread("/content/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/train/happy/387249.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))

im = test_image.astype(np.float32)
im = np.expand_dims(im, axis = 0)

In [ ]:
%%time
onnx_pred = m.run(output_names, {"input": im}) #im is our input image

CPU times: user 351 ms, sys: 1 ms, total: 352 ms
Wall time: 355 ms


Slightly better but main improvement is the memory used. You can quantize on the GPU as well but its more difficult, and improvements are Lets check the accuracy

In [ ]:
def accuracy(model):
  total, acc = 0,0
  for im, label in val_dataset:
    onnx_pred = model.run(output_names, {"input": np.array(im)})

    if(int(np.argmax(onnx_pred, axis = -1)[0][0]) == int(np.argmax(label, axis = -1)[0])):
      acc += 1

    total += 1
  return acc/total

In [ ]:
providers=['CPUExecutionProvider']
m = rt.InferenceSession("/content/vit_keras.onnx", providers=providers)
m_q = rt.InferenceSession("/content/vit_quantized.onnx", providers=providers)
print(accuracy(m_q))
print(accuracy(m))


0.7906057945566286
0.7906057945566286


You can also visualize onnx models using Lutz Rodas Neutron app

**Quantization Aware Training**

We use tensorflow model optimization for quantization (tfmot)

In [ ]:
!pip install -q tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 11.2 MB/s eta 0:00:00


In [ ]:
import tensorflow_model_optimization as tfmot

We willquantize the efficient net model

In [ ]:
backbone = tf.keras.applications.efficientnet.EfficientNetB5(
    include_top = False,
    weights='imagenet',
    input_shape=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3),
)

backbone.trainable = False

x = GlobalAveragePooling2D()(backbone.output)
x = Dense( CONFIGURATION["N_DENSE_1"], activation = "relu")(x)
x = BatchNormalization()(x)
x = Dense( CONFIGURATION["N_DENSE_2"], activation = "relu")(x)
output = Dense( CONFIGURATION["NUM_CLASSES"], activation = "softmax")(x)

pretrained_func_model = tf.keras.Model(inputs = backbone.input, outputs = output )

pretrained_func_model.summary()

115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_12 (Rescaling)  │ (None, 256, 256, 3)    │              0 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization_6           │ (None, 256, 256, 3)    │              7 │ rescaling_12[0][0]     │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_13 (Rescaling)  │ (None, 256, 256, 3)    │              0 │ normalization_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 257, 257, 3)    │              0 │ rescaling_13[0][0]     │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 128, 128, 48)   │          1,296 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 128, 128, 48)   │            192 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 128, 128, 48)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 128, 128, 48)   │            432 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 128, 128, 48)   │            192 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 128, 128, 48)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 48)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 48)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 12)       │            588 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 30,747,386 (117.29 MB)

 Trainable params: 2,231,811 (8.51 MB)

 Non-trainable params: 28,515,575 (108.78 MB)

Now quantize it. We cannot quantize the normalization layers, so we have to select layer by layer and make quantization aware

If we have no batch norm layers can run the below code.

Use quantize_annotate_layer method

In [ ]:
# quant_aware = tfmot.quantization.keras.quantize_model(pretrained_func_model)

ValueError: `to_quantize` can only either be a keras Sequential or Functional model.

In [ ]:
def apply_quantization_to_conv(layer):
  if "conv" in layer.name:
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

Now we will clone our model to be one where quanization is aware, we can clone models and then change it by applying a specific function. This is not working for efficient net for some reason, but will work in general.

In [ ]:
quant_aware_eff = tf.keras.models.clone_model(
    pretrained_func_model,
    clone_function = apply_quantization_to_conv)

quant_aware_eff.summary()

ValueError: `to_annotate` can only be a `keras.layers.Layer` instance. You passed an instance of type: ZeroPadding2D.

In [ ]:
quant_aware_model = tfmot.quantization.keras.quantize_apply(quant_aware_eff)
quant_aware_model.summary()

**Post training Quantization with TF Lite**

TensorFlow Lite library is used to deploy machine learning models on mobile devices, microcontrollers, and edge devices. This is used on drones, and arduinos and other places with low computing power.

This representative generator function yields a smaller portion of the dataset to be used in quantization as static quantization requires to be calibrated on a small dataset.

In [ ]:
def representative_data_gen():
  for input_value, j in training_dataset.take(20):
    yield [input_value]

If we wanted to do dynamic quantization we wouldn't specify the representative data generator

We initialize the tensorflow lite converter and use a function that takes a keras model, on the first line. We use the default optimizations, that applies quantization, and then specify the input and output types. Then we finally convert.

In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model(hf_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen

tflite_model = converter.convert()

Saved artifact at '/tmp/tmpsqp1yj6l'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name='keras_tensor_3472')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  132760189554000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132760189778432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132762610503344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132760189659872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132760190440672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132760190438384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132760189779664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132760189765584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132760189774912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132760189776848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132760189

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


The path we want to save our tf model. The size will be now 21MB, very small, efficient. Use the write bytes method in Pathlib to write the individual bytes

In [ ]:
import pathlib

tflite_models_dir = pathlib.Path("/content/quantized_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"eff_model.tflite"
tflite_model_file.write_bytes(tflite_model)


Once we have the tensorflow lite file, if we want to run this in another system like a raspberry pi, just need to install the tflite_runtime, no need tensorflow.

In [ ]:
!pip3 install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

In [ ]:

import tflite_runtime as tflite
import numpy as np
import cv2

test_image = cv2.imread("/content/human-emotions-datasethes/Emotions Dataset/Emotions Dataset/train/happy/148266.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))
test_image = np.expand_dims(test_image, axis = 0)

Load a tensorflow lite interpreter to interpret the model, this will allow us to do inference. The next line allocates the memory necessary for the tensors, the input and output arrays. It sets the input tensor to be the test image data which is already in the correct format, we need the input index to set it to the correct index. The invoke function runs inference. And then we get the value from the output.

In [ ]:
interpreter = tflite.Interpreter(model_path="/content/drive/MyDrive/Bang/eff_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0] #uint8
output_details = interpreter.get_output_details()[0] #uint8
test_image = im.numpy().astype(input_details["dtype"]) #convert to uint8

interpreter.set_tensor(input_details["index"], test_image)
interpreter.invoke()

output = interpreter.get_tensor(output_details["index"])[0]

Obtain the category

In [ ]:
print(CONFIGURATION['CLASS_NAMES'][np.argmax(output)])

Now lets measure the accuracy of this tensorflow lite model and see if we have lost accuracy. Basically the same as above but with the entire dataset.

In [ ]:
def accuracy(model_path):
  total, correct = 0,0
  interpreter = tf.lite.Interpreter(model_path=model_path)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]


  for im, label in val_dataset:

    test_image = im.numpy().astype(input_details["dtype"])

    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    if(int(np.argmax(output)) == int(np.argmax(label, axis = -1)[0])):
      correct += 1

    total += 1
  return correct/total

Only a slight drop in accuracy, but the model can be deployed in a mobile.

#Model Deployment to APIs

To make some app access the model to feed inputs and receive outputs, we need to create an application programming interfaces. To access APIs, HTTP is typically used. Everytime a client needs to communicate, a connection is created but once that is done, the connection is closed.

Use the HTTPS request library

We can use FastAPI framework to build an API, other frameworks include Flask and Django. FastAPI has become more popular recently.

In [1]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.2 MB/s eta 0:00:00


In [2]:
import fastapi
fastapi.__version__

'0.115.0'

Now different projects may need different fast api version so then use python environments.

In [3]:
!apt install python3-venv

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.10-venv
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3-venv python3.10-venv
0 upgraded, 4 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,475 kB of archives.
After this operation, 2,891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-pip-whl all 22.0.2+dfsg-1ubuntu0.4 [1,680 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-setuptools-whl all 59.6.0-1.2ubuntu0.22.04.2 [788 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3.10-venv amd64 3.10.12-1~22.04.6 [5,722 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python3-venv amd64 3.10.6-1~22.04.1 [1,042 B]
Fetched 2,475 kB in 1s (2

Create the venv, you can create it with a specific version of python (3 or 3.2, 3.8 or whateveer). And to check the version just type python

In [12]:
!python3 -m venv venv-emotions-detection

In [13]:
!ls

sample_data  venv-emotions-detection


In [22]:
%cd venv-emotions-detection

/content/venv-emotions-detection


In [23]:
!ls

bin  include  lib  lib64  pyvenv.cfg


In [24]:
%cd bin

/content/venv-emotions-detection/bin


In [25]:
!ls

activate      activate.fish  pip   pip3.10  python3
activate.csh  Activate.ps1   pip3  python   python3.10


In [26]:
%cd ..

/content/venv-emotions-detection


In [35]:
!source bin/activate

This may not work as expected in colab but this will be how it works in the command line, install the fast api version here.

In [36]:
#Deactivate
!deactivate

/bin/bash: line 1: deactivate: command not found


Now install uvicorn, which is an ASCII server. Uvicorn http server takes requests from users, interacts with fast api, and returns the response. The code is available from the fast api tutorials.

We create an instance of the FASTAPI application and assign it to the variable app, which will handle incoming requests. Next we define a route (endpoint), the app variable takes in requests and maps them to endpoints. The decorator defines a GET route at the root path "/", so whenever the server receives an HTTP GET request, it will call the read_root function sending a JSON response back to the client.

If you run the code and go to http://localhost:8000/ port 8000, then you will see {"Hello": "World"}. This is a very simple way to create an endpoint. The root is a starting point (after the / in the URL) of a website or API.

In [2]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
  return {"Hello": "World"}

To run the server do (in the same folder as your api code)

In [3]:
!uvicorn filename:app

/bin/bash: line 1: uvicorn: command not found


If you want to specify a root directory such that if someone puts http://localhost:8000/something then in the decorator you should have @app.get("/something")

In [6]:
!uvicorn filename:app --reload
#automatically reloads the server instead of manually doing it everytime you change something

/bin/bash: line 1: uvicorn: command not found


you can also go to localhost:8000/docs to see infomation about the server, input arguments etc. or /edoc for an alternative UI.

The following code allows users to retrieve information about a specific item by providing an item id, and a query parameter q (optional). Union allows a variable to have multiple data types (here str or None). The item id has to be part of the path parameter http://localhost:8000/items/42, can include a query as well with http://localhost:8000/items/42?q=fastapi where the query is "fastapi" in this case

In [4]:
from typing import Union

@app.get("/items/{item_id}")
def read_item(item_id: int, q: Union[str, None] = None):
  return {"item_id": item_id, "q": q}

Schemas are a way to define the structure of the data the API will receive or return, which is represented using Pydantic Models. Here InputItem is a Pydantic model (schema) that defines the structure of data expected when an item is added. OutputItem is a schema defining the structure of data returned by the API. If the data types do not match, an HTTP error code will be sent.

We use a post decorator, the output response model is of type output item. Post decorate route is used to handle post requests where client needs to send data to a server to create or modify a resource. GET request parameters are passed in the URL, whereas POST requests send data in the body of the request, so more complex data can be transferred.

In [5]:
from pydantic import BaseModel

class InputItem(BaseModel):
  name: str
  price: int
  discount: int

class OutputItem(BaseModel):
  name: str
  selling_price: int

@app.post("/items/", response_model = OutputItem)
def add_item(item: InputItem):
  selling_price = item.price - item.discount

  return {"name": item.name, "selling_price": selling_price}

Typically we include the schemas, core logic, the endpoints in different files.

In [7]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.0 MB/s eta 0:00:00


In [8]:
import onnxruntime
import cv2

Run the code and inspect the colab folders to see the directory structure

In [9]:
%mkdir service
%cd service
%mkdir api
%mkdir core
%cd api
%mkdir endpoint

/content/service
/content/service/api


create detect.py and test.py in endpoints folder

In [ ]:
!pip install pillow

detect.py detects the emotions. It posts an uploaded file, checks if its an image file, opens the file in bytes form then passes it through the detector.

In [ ]:
#detect.py

from fastapi import APIRouter, UploadFile, HTTPException
from PIL import Image
from io import BytesIO
import numpy as np
from service.core.logic.onnx_inference import emotions_detector
from service.core.schemas.output import APIOutput

emo_router = APIRouter()

@emo_router.post("/detect", response_model=APIOutput)
def detect(im: UploadFile):

  if im.filename.split(".")[-1] in ["jpg", "jpeg", "png"]:
    pass
  else:
    raise HTTPException(status_code=415, detail="Not image")
  image = Image.open(BytesIO(im.file.read()))
  image = np.array(image)

  return emotions_detector(im)


test py just checks if everything is working correctly.

In [ ]:
#test.py

from fastapi import APIRouter, UploadFile, HTTPException
from PIL import Image
from io import BytesIO
import numpy as np

test_router = APIRouter()

@test_router.get("/test")
def testing():

  return {"testing": "testing"}

Now create another file but in the api folder but not in the endpoint folder called api.py. We will create the main router which will use the test and detect router.

The api file combines the detecting and test router into one main router.

In [ ]:
#api.py

from fastapi import APIRouter
from service.api.endpoints.detect import emo_router
from service.api.endpoints.test import test_router

main_router=APIRouter()

main_router.include_router(emo_router)
main_router.include_router(test_router)

We need to turn the api folder and endpoint folder into packages so in both of them create a file called __init__.py. Same with the core folder create 2 more folders called logic and schemas and create an init file inside both, and an init file inside core, and finally an init file inside service so these are all packages.

In the logic folder create a file called onnx_inference.py. Put the onnx file in the logic folder.

Back to main.py which is outside everything

In [ ]:
#main.py

from fastapi import FastAPI
from service.api.api import main_router

app = FastAPI(project_name="emotions detection")

app.include_router(main_router)

@agg.get("/")
def root():
  return {"Hello World": "Welcome to the emotions detection API"}

In [ ]:
#onnx_inference.py

import onnx_runtime as rt
import cv2
import numpy as np

def emotions_detector(test_image):

  if len(test_image) == 2:
    #if grayscale convert to rgb
    test_image = cv2.cvtColor(test_image, cv2.COLOR_GRAY2RGB)

  providers=['CPUExecutionProvider']
  m_q = rt.InferenceSession("service/vit_quantized.onnx", providers=providers)

  test_image = cv2.resize(test_image, (256, 256))

  im = test_image.astype(np.float32)
  img_array = np.expand_dims(im, axis = 0)

  onnx_pred = m_q.run(['dense'], {"input": img_array})

  if np.argmax(onnx_pred[0][0]) == 0:
    emotion = "angry"
  elif np.argmax(onnx_pred[0][0]) == 1:
    emotion = "happy"
  elif np.argmax(onnx_pred[0][0]) == 2:
    emotion = "sad"

  return {"emotion": emotion}

Now inside core create another folder called schemas with files input.py and output.py

In [ ]:
from pydantic import BaseModel

class APIOutput(BaseModel):
  emotion: str
  #include code in onnx_inference to calculate and return time for loading and inference
  time_elapsed: str
  time_elapsed_loading: str

Instead of loading the image everytime we make a call to the API. How about we include the inference session in main.py. Then in the inference file do import service.main as s then m_q becomes s.m_q. Now it is twice as fast.

Now go to localhost:8000/docs and we can upload a file and our model is deployed.

You can also build asynchronous apis in fast api by just putting async before the root function. This means tasks can be run asynchronously so we run multiple tasks while one is still running. But sometimes even if we run asynchronously and complete other tasks, the main one may still throttle the process. So in computer vision, machine learning we utilize parallelism.

Uvicorn is fast but not mature enough to be used in production settings, use Gunicorn or something instead.

In [ ]:
!pip install gunicorn
!gunicorn service.main:app --workers 3 --worker-class uvicorn.workers.UvicornWorker

#Deploying API to the Cloud

Use heroku to deploy the api to the cloud.

First lets put all the packages and versions we have in our venv into a requirements.txt file

In [1]:
!pip freeze > requirements.txtc

In [4]:
with open("requirements.txt", "r") as f:
  x = f.readlines()

x

['absl-py==1.4.0\n',
 'accelerate==0.34.2\n',
 'aiohappyeyeballs==2.4.0\n',
 'aiohttp==3.10.5\n',
 'aiosignal==1.3.1\n',
 'alabaster==0.7.16\n',
 'albucore==0.0.16\n',
 'albumentations==1.4.15\n',
 'altair==4.2.2\n',
 'annotated-types==0.7.0\n',
 'anyio==3.7.1\n',
 'argon2-cffi==23.1.0\n',
 'argon2-cffi-bindings==21.2.0\n',
 'array_record==0.5.1\n',
 'arviz==0.19.0\n',
 'astropy==6.1.3\n',
 'astropy-iers-data==0.2024.9.16.0.32.21\n',
 'astunparse==1.6.3\n',
 'async-timeout==4.0.3\n',
 'atpublic==4.1.0\n',
 'attrs==24.2.0\n',
 'audioread==3.0.1\n',
 'autograd==1.7.0\n',
 'babel==2.16.0\n',
 'backcall==0.2.0\n',
 'beautifulsoup4==4.12.3\n',
 'bigframes==1.18.0\n',
 'bigquery-magics==0.2.0\n',
 'bleach==6.1.0\n',
 'blinker==1.4\n',
 'blis==0.7.11\n',
 'blosc2==2.0.0\n',
 'bokeh==3.4.3\n',
 'bqplot==0.12.43\n',
 'branca==0.7.2\n',
 'build==1.2.2\n',
 'CacheControl==0.14.0\n',
 'cachetools==5.5.0\n',
 'catalogue==2.0.10\n',
 'certifi==2024.8.30\n',
 'cffi==1.17.1\n',
 'chardet==5.2.0\n',
 '

In [ ]:
!sudo snap install heroku
!heroku login

#Follow instructions on heroku tutorial

We can use locust to test the API. It is load testing which tests how well api reacts to millions of users, and measures how many people are using the api.

In [ ]:
from locust import SequentialTaskSet, HttpUser, task

class DetectorTask(SequentialTaskSet):
  @task
  def detection(self):
    with open("test_image.jpg", "rb") as image:
      self.client.post(
          "/detect",
          files = {"im": image}
      )

class LoadTester(HttpUser):
  host=f"{heroku_link}"
  tasks = [DetectorTask]

Launch the locust

In [ ]:
!pip install locust
!locust -f filename.py